In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
from MulticoreTSNE import MulticoreTSNE as TSNE
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

initialize(version_base=None, config_path="configs")
cfg = compose(config_name="config.yaml")

hydra.initialize()

/mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4. WGAN-GP/test2/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [5]:
from dotenv import load_dotenv
import wandb
import os

load_dotenv()
wandb.login(key=os.getenv("WANDB_KEY"))

api = wandb.Api()

True

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: g-hojatkarami (hokarami). Use `wandb login --relogin` to force relogin


True

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from tqdm import tqdm

# Load Data

In [9]:
pd.options.mode.copy_on_write

False

In [10]:
from data.data_utils import get_datasets

train_dataset, val_dataset = get_datasets(
        cfg.data, split=cfg.split, preprocess=True
    )

Discarded 7 ids


preprocessing temporal data into images:   9%|▉         | 887/9593 [00:02<00:29, 296.45it/s]


KeyboardInterrupt: 

# Train/Load Model

In [21]:
from Utils.models import TimEHR

model = TimEHR(cfg)

Generator has 1,304,964 trainable parameters
Critic has 212,177 trainable parameters
Generator has 54446273 parameters
Discriminator has 39593 parameters


In [22]:
# model.from_pretrained(path_cwgan='hokarami/CWGAN/uj5gf643',path_pix2pix='hokarami/PIXGAN/lorajx1i')
model.from_pretrained(path_cwgan='hokarami/CWGAN/uj5gf643',path_pix2pix='hokarami/PIXGAN/9hec4rye')


INFO:root:Loading CWGAN from run: hokarami/CWGAN/uj5gf643


CWGAN - run_path:  hokarami/CWGAN/uj5gf643


INFO:root:Loading PIXGAN from run: hokarami/PIXGAN/9hec4rye


run is old
last_epoch:  400
PIXGAN - run_path:  hokarami/PIXGAN/9hec4rye
last_epoch:  150


In [ ]:
model.train(train_dataset, val_dataset)

In [23]:
model.cwgan.run_path
model.cwgan.epoch_no

model.pix2pix.run_path
model.pix2pix.epoch_no

model.progress

'hokarami/CWGAN/uj5gf643'

400

'hokarami/PIXGAN/9hec4rye'

150

{}

# Generation

In [40]:
from Utils.utils import mat2df
# get synthetic data
fake_static, fake_data = model.generate(train_dataset, count=len(train_dataset), method='real')
df_ts_fake, df_static_fake = mat2df(fake_data,fake_static, train_dataset.dynamic_processor, train_dataset.static_processor)

# get train data
train_static, train_data = model._get_data(train_dataset)
df_ts_train, df_static_train = mat2df(train_data,train_static, train_dataset.dynamic_processor, train_dataset.static_processor)

# get test data
val_static, val_data = model._get_data(val_dataset)
df_ts_test, df_static_test = mat2df(val_data,val_static,val_dataset.dynamic_processor,val_dataset.static_processor)

INFO:root:Using static data from the training set (9593 samples)
100%|██████████| 38/38 [00:00<00:00, 380.08it/s]


# Evaluation

In [42]:
from Utils.evaluation import evaluate

inputs = {

    # normalized data
    'fake_static': fake_static,
    'fake_data': fake_data,
    'train_static': train_static,
    'train_data': train_data,
    'test_static': val_static,
    'test_data': val_data,

    # dataframes from normalized data
    'df_ts_fake': df_ts_fake,
    'df_static_fake': df_static_fake,
    'df_ts_train': df_ts_train,
    'df_static_train': df_static_train,
    'df_ts_test': df_ts_test,
    'df_static_test': df_static_test,

    'state_vars': train_dataset.temporal_features,
}

In [43]:
fake_data.shape, train_data.shape, val_data.shape

((9593, 2, 64, 64), (9593, 2, 64, 64), (2394, 2, 64, 64))

In [45]:
wandb_task_name = f"NOTEBOOK-{cfg.data.name}-s{cfg.split}"

evaluate(inputs, wandb_task_name)

# You can now view the results in the wandb dashboard under USERNAME/TimEHR_Results
# The results are also available locally in Results/{wandb_task_name}/wandb/{run_name}/

[info] Preprocessing 
[info] Preprocessing 
[info] Preprocessing 
[info] Preprocessing 

[info] TRTR
[LightGBM] [Info] Number of positive: 1376, number of negative: 8217
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013864 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28291
[LightGBM] [Info] Number of data points in the train set: 9593, number of used features: 175
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.143438 -> initscore=-1.787024
[LightGBM] [Info] Start training from score -1.787024
AUROC: 0.8316548363996352
AUPRC: 0.4755285238145955
F1: 0.41573033707865165

[info] TSTR
[LightGBM] [Info] Number of positive: 1376, number of negative: 8217
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012545 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 42675
[LightGBM] [Info] Number of data points in the trai

KeyError: "['Label'] not found in axis"